In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# TP1.1 : Scraping du contenu statique 

In [2]:
url = 'https://fstt.ac.ma/Portail2023/'

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive"
}

In [3]:
results_static = pd.DataFrame(columns=['title','link', 'date','type'])

In [4]:
try :
    response = requests.get(url,headers=headers)
    response.raise_for_status()  # Vérifie si la requête a réussi (code 200)
except requests.exceptions.RequestException as e:
    print(f"Erreur lors de la requête : {e}")
    exit()
else :
    soup = BeautifulSoup(response.content, "html.parser")

    # find les titres h2 
    for article in soup.find_all('article',class_='elementor-post'):
        row = {'title':article.find('a').get_text().strip(),"link": article.find('a')['href'], "date": article.find('span',class_='elementor-post-date').get_text().strip(),'type':'BeautifulSoup'}
        results_static = pd.concat([results_static, pd.DataFrame([row])], ignore_index=True)
        
    time.sleep(5)

In [5]:
results_static

,title,link,date,type
0,11ÈME ÉDITION DU CONCOURS FRANCOPHONE INTERNAT...,https://fstt.ac.ma/Portail2023/11eme-edition-d...,"avril 8, 2025",BeautifulSoup
1,APPEL À PARTICIPATION AU CLUB « SHABAB LABS – ...,https://fstt.ac.ma/Portail2023/appel-a-partici...,"avril 3, 2025",BeautifulSoup
2,INVITATION : JOURNÉE DOCTORALE,https://fstt.ac.ma/Portail2023/invitation-jour...,"avril 2, 2025",BeautifulSoup
3,FORMATION À DISTANCE EN « TÉLÉDÉTECTION RADAR ...,https://fstt.ac.ma/Portail2023/formation-a-dis...,"avril 1, 2025",BeautifulSoup


In [6]:
# saving results to a CSV file
results_static.to_csv('fstt_articles_tp1_1.csv', index=False, encoding='utf-8-sig')

# TP1.2 : Scraping du contenu dynamique

In [22]:

chromedriver_path = 'C:/tools/chromedriver.exe'

# Configuration des options Chrome
options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # Exécution en mode sans tête
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')  # Désactiver le GPU, utile pour les environnements virtuels
options.add_argument('--verbose')  # Ajoute cette ligne pour plus de détails dans les logs


# Créer le service avec le chemin du chromedriver
service = Service(chromedriver_path)

# Initialiser le WebDriver avec le service
driver = webdriver.Chrome(service=service, options=options)
title = None

try:
    driver.get('https://fstt.ac.ma/Portail2023/')
    print(driver.title)
    # Extraire les articles
    articles = driver.find_elements(By.TAG_NAME, 'article')

    # Créer une liste pour stocker les résultats
    results = []

    for article in articles:
        title = article.find_element(By.TAG_NAME, 'a').text.strip() if title else 'Titre non disponible'
        # Extraire le lien de l'article
        link = article.find_element(By.TAG_NAME, 'a').get_attribute('href')
        
        # Extraire la date de l'article
        date_element = article.find_element(By.CSS_SELECTOR, 'span.elementor-post-date')
        date = date_element.text.strip() if date_element else 'Date non disponible'
        print('title : ',title,' link : ',link,' date : ',date)
        
        # Ajouter les données dans la liste des résultats
        results.append({"title":title,"link": link, "date": date,"type":'selenium'})
except Exception as e:
    print(f"Une erreur s'est produite : {e}")


FST – TANGER
title :  Titre non disponible  link :  https://fstt.ac.ma/Portail2023/11eme-edition-du-concours-francophone-international-ma-these-en-180-secondes-edition-2025/  date :  avril 8, 2025
title :  APPEL À PARTICIPATION AU CLUB « SHABAB LABS – JEUNES EN TANT QUE CHERCHEURS » DE L’UNESCO  link :  https://fstt.ac.ma/Portail2023/appel-a-participation-au-club-shabab-labs-jeunes-en-tant-que-chercheurs-de-lunesco/  date :  avril 3, 2025
title :  INVITATION : JOURNÉE DOCTORALE  link :  https://fstt.ac.ma/Portail2023/invitation-journee-doctorale/  date :  avril 2, 2025
title :  FORMATION À DISTANCE EN « TÉLÉDÉTECTION RADAR : MISSION SENTINEL-1 »  link :  https://fstt.ac.ma/Portail2023/formation-a-distance-en-teledetection-radar-mission-sentinel-1/  date :  avril 1, 2025


In [23]:

# Convertir la liste des résultats en DataFrame
df = pd.DataFrame(results)
df

,title,link,date,type
0,Titre non disponible,https://fstt.ac.ma/Portail2023/11eme-edition-d...,"avril 8, 2025",selenium
1,APPEL À PARTICIPATION AU CLUB « SHABAB LABS – ...,https://fstt.ac.ma/Portail2023/appel-a-partici...,"avril 3, 2025",selenium
2,INVITATION : JOURNÉE DOCTORALE,https://fstt.ac.ma/Portail2023/invitation-jour...,"avril 2, 2025",selenium
3,FORMATION À DISTANCE EN « TÉLÉDÉTECTION RADAR ...,https://fstt.ac.ma/Portail2023/formation-a-dis...,"avril 1, 2025",selenium


In [24]:
# Sauvegarder les résultats dans un fichier CSV
df.to_csv("fstt_articles_tp1_2.csv", index=False)

# Conacination of results

In [25]:
import pandas as pd


# Fusionner les deux DataFrames
combined_df = pd.concat([results_static, df], ignore_index=True)

# Sauvegarder les données combinées dans un fichier CSV
combined_df.to_csv('articles_combined.csv', index=False)

print("Données combinées et sauvegardées dans 'articles_combined.csv'")
combined_df


Données combinées et sauvegardées dans 'articles_combined.csv'


,title,link,date,type
0,11ÈME ÉDITION DU CONCOURS FRANCOPHONE INTERNAT...,https://fstt.ac.ma/Portail2023/11eme-edition-d...,"avril 8, 2025",BeautifulSoup
1,APPEL À PARTICIPATION AU CLUB « SHABAB LABS – ...,https://fstt.ac.ma/Portail2023/appel-a-partici...,"avril 3, 2025",BeautifulSoup
2,INVITATION : JOURNÉE DOCTORALE,https://fstt.ac.ma/Portail2023/invitation-jour...,"avril 2, 2025",BeautifulSoup
3,FORMATION À DISTANCE EN « TÉLÉDÉTECTION RADAR ...,https://fstt.ac.ma/Portail2023/formation-a-dis...,"avril 1, 2025",BeautifulSoup
4,Titre non disponible,https://fstt.ac.ma/Portail2023/11eme-edition-d...,"avril 8, 2025",selenium
5,APPEL À PARTICIPATION AU CLUB « SHABAB LABS – ...,https://fstt.ac.ma/Portail2023/appel-a-partici...,"avril 3, 2025",selenium
6,INVITATION : JOURNÉE DOCTORALE,https://fstt.ac.ma/Portail2023/invitation-jour...,"avril 2, 2025",selenium
7,FORMATION À DISTANCE EN « TÉLÉDÉTECTION RADAR ...,https://fstt.ac.ma/Portail2023/formation-a-dis...,"avril 1, 2025",selenium


# Bonus : 

##  Gestion de la pagination dynamique:

In [ ]:
article_results = []
title = None
# Boucle pour gérer la pagination
while True:
    # Extraire les articles de la page actuelle
    articles = driver.find_elements(By.TAG_NAME, 'article')
    for article in articles:
        title = article.find_element(By.TAG_NAME, 'a').text.strip() if title else 'Titre non disponible'
        # Extraire le lien de l'article
        link = article.find_element(By.TAG_NAME, 'a').get_attribute('href')
        
        # Extraire la date de l'article
        date_element = article.find_element(By.CSS_SELECTOR, 'span.elementor-post-date')
        date = date_element.text.strip() if date_element else 'Date non disponible'
        print('title : ',title,' link : ',link,' date : ',date)
        
        # Ajouter les données dans la liste des résultats
        article_results.append({"title":title,"link": link, "date": date})
    # Vérifier si le bouton "Suivant" existe et cliquer dessus
    
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "span.elementor-button-text")
        if next_button.text != "Voir plus":
            print("Le bouton 'Voir plus' n'est pas disponible.")
            print(next_button.text)
            break
        next_button.click()  # Cliquer sur "Suivant"
        time.sleep(5)  # Attendre que la page suivante charge
    except:
        print("Fin de la pagination")
        break


title :  Titre non disponible  link :  https://fstt.ac.ma/Portail2023/11eme-edition-du-concours-francophone-international-ma-these-en-180-secondes-edition-2025/  date :  avril 8, 2025
title :  APPEL À PARTICIPATION AU CLUB « SHABAB LABS – JEUNES EN TANT QUE CHERCHEURS » DE L’UNESCO  link :  https://fstt.ac.ma/Portail2023/appel-a-participation-au-club-shabab-labs-jeunes-en-tant-que-chercheurs-de-lunesco/  date :  avril 3, 2025
title :  INVITATION : JOURNÉE DOCTORALE  link :  https://fstt.ac.ma/Portail2023/invitation-journee-doctorale/  date :  avril 2, 2025
title :  FORMATION À DISTANCE EN « TÉLÉDÉTECTION RADAR : MISSION SENTINEL-1 »  link :  https://fstt.ac.ma/Portail2023/formation-a-distance-en-teledetection-radar-mission-sentinel-1/  date :  avril 1, 2025
title :  11ÈME ÉDITION DU CONCOURS FRANCOPHONE INTERNATIONAL « MA THÈSE EN 180 SECONDES » – EDITION 2025  link :  https://fstt.ac.ma/Portail2023/11eme-edition-du-concours-francophone-international-ma-these-en-180-secondes-edition-20

In [18]:
article_results = pd.DataFrame(article_results)

# remove duplicates
article_results = article_results.drop_duplicates(subset=['title'], keep='first')

article_results

,title,link,date
0,Titre non disponible,https://fstt.ac.ma/Portail2023/11eme-edition-d...,"avril 8, 2025"
1,APPEL À PARTICIPATION AU CLUB « SHABAB LABS – ...,https://fstt.ac.ma/Portail2023/appel-a-partici...,"avril 3, 2025"
2,INVITATION : JOURNÉE DOCTORALE,https://fstt.ac.ma/Portail2023/invitation-jour...,"avril 2, 2025"
3,FORMATION À DISTANCE EN « TÉLÉDÉTECTION RADAR ...,https://fstt.ac.ma/Portail2023/formation-a-dis...,"avril 1, 2025"
4,11ÈME ÉDITION DU CONCOURS FRANCOPHONE INTERNAT...,https://fstt.ac.ma/Portail2023/11eme-edition-d...,"avril 8, 2025"
8,16ÈME ÉDITION DU CONCOURS NATIONAL DE L’INNOVA...,https://fstt.ac.ma/Portail2023/16eme-edition-d...,"mars 18, 2025"
9,APPEL À CANDIDATURE : PROGRAMME DE VISITES DE ...,https://fstt.ac.ma/Portail2023/appel-a-candida...,"mars 11, 2025"
10,8éme CONFÉRENCE INTERNATIONALE SUR LES RÉSEAUX...,https://fstt.ac.ma/Portail2023/8eme-conference...,"mars 11, 2025"
11,DATA2025 HACKATHON,https://fstt.ac.ma/Portail2023/data2025-hackat...,"mars 11, 2025"
12,تعليق الدراسة يوم الإثنين بسبب اضطرابات الأحوا...,https://fstt.ac.ma/Portail2023/%d8%aa%d8%b9%d9...,"mars 9, 2025"


In [19]:
# Sauvegarder les résultats dans un fichier CSV
article_results.to_csv("fstt_articles_tp1_bonus_1.csv", index=False)

## Extraction des images ou autres médias

In [20]:
images = driver.find_elements(By.TAG_NAME, 'img')
image_urls = [img.get_attribute('src') for img in images]

images = pd.DataFrame(image_urls, columns=['image_url'])

# Sauvegarder les URLs des images dans un fichier CSV
images.to_csv('images.csv', index=False, encoding='utf-8-sig')

print("URLs des images extraites et sauvegardées dans 'images.csv'")


driver.quit()

URLs des images extraites et sauvegardées dans 'images.csv'


In [21]:
images

,image_url
0,https://fstt.ac.ma/Portail2023/wp-content/uplo...
1,https://fstt.ac.ma/Portail2023/wp-content/uplo...
